# Fine-tuning Didático de um Modelo GPT (OSS) com LoRA (PEFT)

Este notebook demonstra, de forma **didática e prática**, como realizar **fine-tuning** de um modelo **GPT open-source** usando a biblioteca **Hugging Face Transformers** em conjunto com **PEFT (LoRA)** para tornar o processo **viável em hardware modesto**.

> **Importante:** Modelos com ~20B de parâmetros **exigem infraestrutura robusta** (GPUs de alta memória, quantização, FSDP/DeepSpeed, etc.). Neste notebook, usamos um **modelo menor** para ilustrar os conceitos e a pipeline completa.


## Objetivos de Aprendizagem
- Compreender o fluxo de fine-tuning para **modelos de linguagem causal (GPT-like)**.
- Preparar dados curtos (instrução → resposta) e **tokenizar** corretamente.
- Aplicar **LoRA (PEFT)** para reduzir custo de ajuste fino.
- Treinar, avaliar com **geração** e salvar o adaptador.
- Carregar o adaptador para **inferência** pós-treino.

Implementação baseada em:
[gpt-oss-(20B)-Fine-tuning](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-Fine-tuning.ipynb#scrollTo=waDcYbdVUesj)

## Requisitos e Instalação

Execute a célula abaixo **se** ainda não possuir as dependências instaladas. 
O conjunto mínimo inclui:
- `transformers`
- `datasets`
- `accelerate`
- `peft`
- `evaluate` (opcional, para métricas)
- `bitsandbytes` (opcional; útil para quantização em GPU NVIDIA)

> Observação: se estiver no Google Colab, selecione **Runtime → Change runtime type → GPU**.


In [ ]:
# Instalação (descomente se necessário)
# !pip install -U transformers datasets accelerate peft evaluate
# !pip install bitsandbytes  # (opcional; funciona com GPUs NVIDIA)


## Verificação de Dispositivo (CPU/GPU)

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")

print("Dispositivo:", device)
if device == "cuda":
    print("Nome da GPU:", torch.cuda.get_device_name(0))



## Escolha do Modelo Base

Para fins didáticos, escolha um **modelo pequeno** compatível com `AutoModelForCausalLM`. 
Alguns exemplos (substitua `model_name` abaixo):
- `gpt2` (inglês, pequeno)
- `uer/gpt2-chinese-cluecorpussmall` (chinês, exemplo multicultural)
- `EleutherAI/pythia-70m-deduped` (família Pythia)
- `openai-community/gpt2` (espelho comunitário do GPT-2)

> Para modelos **maiores** (p.ex., família Falcon/OPT/Mistral/Llama), é recomendada quantização, FSDP/DeepSpeed e múltiplas GPUs.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Modelo base pequeno para fins ilustrativos
model_name = "gpt2"  # altere conforme desejado
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT-2 não possui token pad por padrão; definimos um
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)
print("Modelo carregado.")

## Preparação de Dados (Exemplo Simples)

Abaixo, definimos um **mini dataset** de pares *(instrução → resposta)* em português para ilustrar o processo. 
Em um cenário real, você pode carregar:
- Arquivos JSON/CSV com colunas `instruction` e `output`
- Conjuntos do `datasets` (Hugging Face Hub)
- Seu próprio dataset institucional

Vamos formatar os exemplos como *prompt* concatenado com *resposta*.


In [ ]:
from datasets import Dataset

# Mini dataset ilustrativo (INSTRUÇÃO -> RESPOSTA)
samples = [
    {"instruction": "Explique o que é Aprendizado de Máquina em 1 frase.", 
     "output": "É a área que estuda algoritmos capazes de aprender padrões a partir de dados."},
    {"instruction": "Dê um exemplo de transformação linear.", 
     "output": "Uma matriz multiplicando um vetor no espaço vetorial é um exemplo de transformação linear."},
    {"instruction": "O que é overfitting?", 
     "output": "É quando o modelo memoriza o conjunto de treino e generaliza mal para novos dados."},
    {"instruction": "Defina rede neural.", 
     "output": "É um modelo composto por camadas de neurônios artificiais interconectados que aprendem representações."},
    {"instruction": "Cite um caso de uso de Processamento de Linguagem Natural.", 
     "output": "Análise de sentimento de avaliações de usuários é um caso comum de PLN."},
]

def build_prompt(example):
    return f"Instrução: {example['instruction']}\nResposta:"

def build_target(example):
    return example["output"]

dataset = Dataset.from_list([
    {"text": build_prompt(s) + " " + build_target(s)}
    for s in samples
])

dataset


## Tokenização e Collator

A tokenização converte o texto em IDs de tokens do vocabulário. Para modelos causais (GPT-like), 
o rótulo para treinamento é geralmente o **próprio texto deslocado** (causal LM). 
Usaremos `DataCollatorForLanguageModeling` com `mlm=False`.


In [ ]:
from transformers import DataCollatorForLanguageModeling

def tokenize_function(batch):
    return tokenizer(
        batch["text"], 
        truncation=True, 
        max_length=256, 
        padding=True
    )

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

# Split train/eval simples
split = tokenized.train_test_split(test_size=0.2, seed=42)
train_ds = split["train"]
eval_ds = split["test"]

len(train_ds), len(eval_ds)


## Configuração de PEFT (LoRA)

**LoRA (Low-Rank Adaptation)** permite treinar **apenas pequenos adaptadores** em algumas camadas do modelo, 
reduzindo **drasticamente** o número de parâmetros atualizados e o custo computacional.

Parâmetros comuns:
- `r`: rank da decomposição (ex.: 8, 16, 32)
- `alpha`: escala da atualização (ex.: 16, 32)
- `target_modules`: quais projeções receberão LoRA (ex.: `c_attn`, `q_proj`, `v_proj` dependendo do modelo)


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# Configuração LoRA básica (ajuste conforme o modelo)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn"],  # para GPT-2; para outros modelos pode ser ["q_proj","v_proj"]
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


## Treinamento (Hugging Face Trainer)

Usaremos o `Trainer` para simplificar o loop de treinamento. Ajuste os hiperparâmetros conforme o ambiente.

> Dica: Em GPU com pouca memória, reduza `per_device_train_batch_size`, `max_length` e `r` do LoRA.


In [ ]:
from transformers import TrainingArguments, Trainer
from inspect import signature
import math, torch, transformers

print("Transformers versão:", transformers.__version__)

output_dir = "gpt2-lora-pt"

# kwargs padrão (para versões novas)
ta_kwargs = dict(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=5,
    evaluation_strategy="steps",  # <– pode não existir em versões antigas/carregadas
    eval_steps=10,
    save_steps=20,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),   # só CUDA
    bf16=False,                       # mantenha False no MPS
    report_to="none",
    optim="adamw_torch",
    no_cuda=not torch.cuda.is_available(),
)

# Adaptação automática se a assinatura não tiver 'evaluation_strategy'
params = signature(TrainingArguments).parameters
if "evaluation_strategy" not in params:
    # Remover chaves não suportadas
    ta_kwargs.pop("evaluation_strategy", None)
    ta_kwargs.pop("eval_steps", None)
    # Alternativa para versões antigas: apenas permitir avaliação manual depois
    ta_kwargs["do_eval"] = True
    print("Atenção: 'evaluation_strategy' não suportado nesta classe carregada. Usando fallback com do_eval=True.")

training_args = TrainingArguments(**ta_kwargs)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator
)

train_result = trainer.train()
eval_metrics = trainer.evaluate()

print("Eval metrics:", eval_metrics)
if "eval_loss" in eval_metrics:
    try:
        print("Perplexity:", math.exp(eval_metrics["eval_loss"]))
    except OverflowError:
        print("Perplexity overflow (loss muito grande).")

In [ ]:
import transformers
print(transformers.__version__)

## Avaliação Informal por Geração

Após o fine-tuning, fazemos uma **geração** a partir de uma instrução não vista, 
para verificar se o modelo aprendeu o formato *Instrução → Resposta*.


In [ ]:
from transformers import pipeline

if torch.cuda.is_available():
    device_id = 0
elif hasattr(torch, "mps") and torch.mps.is_available():
    device_id = "mps" 
else:
    device_id = -1  # CPU

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device_id
)

prompt = "Instrução: Explique o que é regularização em aprendizado de máquina.\nResposta:"
gen = pipe(
    prompt,
    max_new_tokens=80,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id
)

print(gen[0]["generated_text"])


## Salvamento e Carregamento do Adaptador LoRA

Salvamos o **adaptador LoRA** (parâmetros aprendidos). Para reutilizar:
1. Carregar o mesmo modelo base.
2. Aplicar `PEFT` com `from_pretrained` no diretório salvo.


In [ ]:
from pathlib import Path
adapter_dir = Path("gpt2-lora-pt") / "lora_adapter"
adapter_dir.mkdir(parents=True, exist_ok=True)

model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)

print("Adaptador LoRA salvo em:", adapter_dir)


## Como usar o adaptador em outro ambiente

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Carregar o adaptador salvo
peft_model = PeftModel.from_pretrained(base_model, "gpt2-lora-pt/lora_adapter")

# Inferência
peft_model.eval()
```

## Depois de carregar o modelo, pode testar com o seguinte trecho
```python
prompt = "Instrução: Explique o que é regularização em aprendizado de máquina.\nResposta:"

inputs = tokenizer(prompt, return_tensors="pt").to(peft_model.device)
outputs = peft_model.generate(
    **inputs,
    max_new_tokens=80,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```
> Em produção, considere quantização (8-bit/4-bit com `bitsandbytes`), FSDP/DeepSpeed e *checkpointing* de ativação para modelos maiores.


## Boas Práticas e Adaptação para Modelos Maiores (~20B)

- **Quantização**: 8-bit/4-bit (`bitsandbytes`) para reduzir memória.
- **Treino distribuído**: FSDP / ZeRO / DeepSpeed para particionar pesos/ótimos/gradientes.
- **Checkpointing de ativação**: reduz memória com custo de recálculo.
- **Gradiente acumulado** + **batch pequeno**: compatibiliza com GPUs menores.
- **Mixed Precision** (`fp16`/`bf16`): acelera e reduz memória (requer suporte de hardware).
- **Data quality**: curadoria dos exemplos (instruções claras, respostas curtas e diretas).
- **Avaliação**: defina *prompts* de validação replicáveis e métricas (p.ex., perplexidade).

> **Infra**: para 20B+, recomenda-se múltiplas GPUs (A100, H100, etc.), armazenamento rápido e orquestração (Slurm, Ray, etc.).
